In [1]:
import pandas as pd

In [5]:
from scipy.stats import skew

In [64]:
data = pd.read_excel('Data/oan_ohlcv.xlsx')
data.set_index('date', inplace=True)

In [69]:
def extend_historicals(historical_data):
    skew_roll = historical_data.rolling(20).apply(skew)
    skew_roll.columns = skew_roll.columns + ' skew'
    historical_data = pd.concat([historical_data, skew_roll], axis=1)
    return skew_roll

In [70]:
new_data = extend_historicals(data)

In [73]:
pd.concat([data, new_data], axis=1)

,AU200_AUD close,AU200_AUD high,AU200_AUD low,AU200_AUD open,AU200_AUD volume,AUD_USD close,AUD_USD high,AUD_USD low,AUD_USD open,AUD_USD volume,...,XPD_USD close skew,XPD_USD high skew,XPD_USD low skew,XPD_USD open skew,XPD_USD volume skew,XPT_USD close skew,XPT_USD high skew,XPT_USD low skew,XPT_USD open skew,XPT_USD volume skew
date,,,,,,,,,,,,,,,,,,,,,
2006-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,7328.7,7372.2,7320.2,7352.2,5100.0,0.66304,0.66684,0.66075,0.66151,49228.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-01,7221.7,7330.2,7181.2,7318.7,13913.0,0.66626,0.67172,0.66208,0.66341,70845.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-02,7160.2,7219.2,7155.2,7219.2,18905.0,0.66728,0.67036,0.66494,0.66627,72404.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
new_data

,AU200_AUD close,AU200_AUD high,AU200_AUD low,AU200_AUD open,AU200_AUD volume,AUD_USD close,AUD_USD high,AUD_USD low,AUD_USD open,AUD_USD volume,...,XPD_USD close,XPD_USD high,XPD_USD low,XPD_USD open,XPD_USD volume,XPT_USD close,XPT_USD high,XPT_USD low,XPT_USD open,XPT_USD volume
date,,,,,,,,,,,,,,,,,,,,,
2006-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,7328.7,7372.2,7320.2,7352.2,5100.0,0.66304,0.66684,0.66075,0.66151,49228.0,...,1449.197,1512.083,1436.962,1498.580,12292.0,1060.742,1099.009,1057.996,1084.240,24608.0
2023-05-01,7221.7,7330.2,7181.2,7318.7,13913.0,0.66626,0.67172,0.66208,0.66341,70845.0,...,1429.517,1457.450,1414.786,1445.454,12047.0,1070.752,1075.075,1048.332,1060.794,27091.0
2023-05-02,7160.2,7219.2,7155.2,7219.2,18905.0,0.66728,0.67036,0.66494,0.66627,72404.0,...,1416.576,1440.528,1402.341,1423.030,10624.0,1061.843,1081.137,1052.820,1070.705,32479.0


In [72]:
new_data

,AU200_AUD close skew,AU200_AUD high skew,AU200_AUD low skew,AU200_AUD open skew,AU200_AUD volume skew,AUD_USD close skew,AUD_USD high skew,AUD_USD low skew,AUD_USD open skew,AUD_USD volume skew,...,XPD_USD close skew,XPD_USD high skew,XPD_USD low skew,XPD_USD open skew,XPD_USD volume skew,XPT_USD close skew,XPT_USD high skew,XPT_USD low skew,XPT_USD open skew,XPT_USD volume skew
date,,,,,,,,,,,,,,,,,,,,,
2006-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30,NaN,NaN,NaN,NaN,NaN,0.114349,0.020176,-0.310996,0.034766,-0.504049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-01,NaN,NaN,NaN,NaN,NaN,0.260575,0.088456,-0.289019,0.117136,-0.471634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-02,NaN,NaN,NaN,NaN,NaN,0.399818,0.216738,-0.215741,0.256657,-0.442123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,AU200_AUD close,AU200_AUD high
date,,
2022-02-07,7196.8,7208.8
2022-05-24,NaN,NaN
2022-05-25,NaN,NaN
2022-05-26,NaN,NaN
2022-05-29,NaN,NaN
...,...,...
2023-04-30,7328.7,7372.2
2023-05-01,7221.7,7330.2
2023-05-02,7160.2,7219.2
